In [73]:
from database import dbconnection
from database.schema  import schema

db = dbconnection.DbConnection()
db.connect()


<Spreadsheet 'Pegasus - DEV' id:1Kd0NMRPhyIN6Ewi2pa4_mmXYUm1D9GYDjcw_7Fcm1TY>

In [77]:

# [{'cat': 'DESPESA FIXA', 'cat_parent': None, 'level': 1, 'value': 'Aluguel'},

,DS_Description,DS_Category
0,COMPRA CARTAO - COMPRA no estabelecimento BK N...,alimentação
1,COMPRA CARTAO - COMPRA no estabelecimento SHOW...,alimentação
2,COMPRA CARTAO - COMPRA no estabelecimento IFD*...,alimentação
3,COMPRA CARTAO - COMPRA no estabelecimento ESTR...,alimentação
4,COMPRA CARTAO - COMPRA no estabelecimento SUBW...,alimentação
...,...,...
138,COMPRA CARTAO - COMPRA no estabelecimento RIOC...,transporte
139,APLICACAO - LCI LIQ 90 DIAS BANCO INTER SA,LCI
140,COMPRA CARTAO - COMPRA no estabelecimento CASA...,eletrônicos
141,COMPRA CARTAO - COMPRA no estabelecimento CARR...,roupas
